# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [1]:
import torch

In [2]:
def getParallelDB(db , removeIndex):
    return(torch.cat((db[:removeIndex] , db[removeIndex+1:])))

In [3]:
def getParallelDatabases(db):
    parallelDatabases = []
    for i in range(len(db)):
        pdb = getParallelDB(db , i)
        parallelDatabases.append(pdb)
    
    return parallelDatabases

In [4]:
def get_createDB_and_parallels(numOfEntries):
    db = torch.rand(numOfEntries)>0.5
    pdbs = getParallelDatabases(db)
    return(db , pdbs)

In [5]:
db, pdbs = get_createDB_and_parallels(100)

In [6]:
trueResult = torch.mean(db.float())

In [7]:
trueResult

tensor(0.4900)

We want to noise our dataset with LocalDifferential Privacy and it is all about adding noise to the data itself
# -
So adding noise to data means replacing some of these values random values 

In [8]:
firstCoinFlip = (torch.rand(len(db)) > 0.5).float()

In [9]:
firstCoinFlip

tensor([1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
        1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
        0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 1., 0., 1., 1.])

In [10]:
secondCoinFlip = (torch.rand(len(db)) > 0.5).float()

In [11]:
secondCoinFlip

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 0.])

If 1st coinFlip is head(that means 1) we keep the values that are inthe database<br>
So we can Multiply the database with firstCoinFlip<br>
After we'll having 0 at the places tail occured<br>
Now I can do (1 - firstCoinFlip) to interchange ones and zeroes and after I can multiply it with secondCoinFlip

In [12]:
db.float() * firstCoinFlip

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 1.])

In [13]:
(1 - firstCoinFlip) * secondCoinFlip

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [14]:
augmentedDatabase = db.float() * firstCoinFlip + (1-firstCoinFlip) * secondCoinFlip

In [15]:
m = torch.mean(augmentedDatabase.float())

In [16]:
# (meanOfDatabase + actual)/2 = augemented
# DeSkew

In [17]:
augmentedResult = (m * 2) - 0.5
augmentedResult

tensor(0.3800)

# Lets Package

In [18]:
def query(db):
    
    trueResult = torch.mean(db.float())
    
    firstCoinFlip = (torch.rand(len(db)) > 0.5).float()
    secondCoinFlip = (torch.rand(len(db)) > 0.5).float()
    
    augmentedDatabase = db.float() * firstCoinFlip + ((1-firstCoinFlip) * secondCoinFlip)
    
    dbResult = torch.mean(augmentedDatabase.float()) * 2 - 0.5
    
    return dbResult , trueResult

In [19]:
query(db)

(tensor(0.5200), tensor(0.4900))

In [20]:
db, pdbs = get_createDB_and_parallels(10)
withNoise, withoutNoise = query(db)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.10000002384185791 And Without Noise 0.5


In [21]:
db, pdbs = get_createDB_and_parallels(100)
withNoise, withoutNoise = query(db)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.42000001668930054 And Without Noise 0.44999998807907104


In [22]:
db, pdbs = get_createDB_and_parallels(1000)
withNoise, withoutNoise = query(db)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.5099999904632568 And Without Noise 0.5059999823570251


In [23]:
db, pdbs = get_createDB_and_parallels(10000)
withNoise, withoutNoise = query(db)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.5146000385284424 And Without Noise 0.4970000088214874


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

# ------------------------------------

In [25]:
noise = 0.2

In [26]:
trueResult = torch.mean(db.float())
    
firstCoinFlip = (torch.rand(len(db)) > noise).float() #Biased coin
secondCoinFlip = (torch.rand(len(db)) > 0.5).float()

augmentedDatabase = db.float() * firstCoinFlip + ((1-firstCoinFlip) * secondCoinFlip)

#There will change in DeSkewing
skResult = torch.mean(augmentedDatabase.float()) 

#return dbResult , trueResult

In [27]:
trueMean = 0.7
noiseMean = 0.5 #second coin flip


In [28]:
#Initially what we are doing is . . .
augmentedMean = (trueMean + noiseMean)/2
augmentedMean

0.6

In [29]:
noise = 0.5 #First coin
#Which actually means 50% of the time we use trueMean and 50% of the time we use noiseMean

In [30]:
augmentedMean = (trueMean*noise) + (noiseMean*(1 - noise))
augmentedMean

0.6

We can get trueMean from above equation by simple algebra

In [31]:
privateResult = (skResult/noise) - ((0.5*(1-noise)) / noise) 

In [32]:
privateResult

tensor(0.5056)

In [33]:
skResult

tensor(0.5028)

# ------------------------------------

In [34]:
def query(db, noise=0.2):
    
    trueResult = torch.mean(db.float())
    
    firstCoinFlip = (torch.rand(len(db)) > noise).float() #Biased coin
    secondCoinFlip = (torch.rand(len(db)) > 0.5).float()
    
    augmentedDatabase = db.float() * firstCoinFlip + ((1-firstCoinFlip) * secondCoinFlip)
    
    #There will change in DeSkewing
    skResult = torch.mean(augmentedDatabase.float()) 
    
    privateResult = (skResult/noise) - ((0.5*(1-noise))/noise)
    return privateResult , trueResult

In [35]:
db, pdbs = get_createDB_and_parallels(100)
withNoise, withoutNoise = query(db, noise=0.1)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.19999980926513672 And Without Noise 0.46000000834465027


In [36]:
db, pdbs = get_createDB_and_parallels(100)
withNoise, withoutNoise = query(db, noise=0.2)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.09999990463256836 And Without Noise 0.44999998807907104


In [37]:
db, pdbs = get_createDB_and_parallels(100)
withNoise, withoutNoise = query(db, noise=0.4)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.6000000238418579 And Without Noise 0.47999998927116394


In [38]:
db, pdbs = get_createDB_and_parallels(100)
withNoise, withoutNoise = query(db, noise=0.8)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.5374999642372131 And Without Noise 0.5600000023841858


In [39]:
db, pdbs = get_createDB_and_parallels(10000)
withNoise, withoutNoise = query(db, noise=0.2)
print("With Noise {} And Without Noise {}".format(withNoise,withoutNoise))

With Noise 0.5504999160766602 And Without Noise 0.5080000162124634


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [40]:
db, pdbs = get_createDB_and_parallels(100)

In [41]:
def query(db):
    return torch.sum(db.float()) + noise
#In localDP we would have done this return torch.sum(db.float() + noise)

In [42]:
query(db)

tensor(50.5000)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

In [43]:
def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

M => is Randomised Algorithm

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [76]:
epsilon = 0.5 # if we decrease this we are actually incresing the amount of noise to be added

In [45]:
import numpy as np

In [53]:
db, pdbs = get_createDB_and_parallels(100)

In [54]:
sum(db)#Max sensitivity can be 1

tensor(53, dtype=torch.uint8)

In [55]:
2*db #Max sensitivity will be 2

tensor([2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2,
        2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0,
        0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,
        2, 0, 0, 0], dtype=torch.uint8)

## Lets jump into query

In [58]:
def sum_query(db):
    return db.sum()

In [71]:
def laplacianMechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    
    noise = torch.tensor(np.random.laplace(0,beta,1)) #Mean centered at zero and beta is controlling the spread
    return query(db) + noise#GDP

In [69]:
laplacianMechanism(db, sum_query, 1)

tensor([53.2056], dtype=torch.float64)

In [75]:
trueSum = db.sum()
trueSum

tensor(53)

### Lets do this for mean query

In [72]:
def mean_query(db):
    return torch.mean(db.float())

Think what can be the sensitivity <br>
Max change divided by total number of entries i.e. 1/len(db) in our case => 1/100

In [73]:
laplacianMechanism(db, mean_query, 1/100)

tensor([0.5401], dtype=torch.float64)

In [74]:
trueMean = torch.mean(db.float())
trueMean

tensor(0.5300)